In [ ]:
from keras.models import Sequential
from keras.models import load_model
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils  import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from utilities import one_hot_encoding, read_train_data, read_label_data

SIZE = 5000
MODEL_FOLDER = "model/"
WIDTH = 200
HEIGHT = 60
PROCESSED_FOLDER = "processed/"
LABEL_CSV_FILE = 'label.csv'
allowedChars = 'ACDEFGHJKLNPQRTUVXYZ2346789';

In [ ]:
# creat CNN model
print('Creating CNN model...')
tensor_in = Input((HEIGHT, WIDTH, 3))
tensor_out = tensor_in

tensor_out = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(tensor_out)
tensor_out = Dropout(0.25)(tensor_out)
tensor_out = MaxPooling2D(pool_size=(2, 2))(tensor_out)

tensor_out = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(tensor_out)
tensor_out = Dropout(0.25)(tensor_out)
tensor_out = MaxPooling2D(pool_size=(2, 2))(tensor_out)

tensor_out = Flatten()(tensor_out)
tensor_out = Dropout(0.25)(tensor_out)

tensor_out = [Dense(len(allowedChars), name='digit1', activation='softmax')(tensor_out),\
              Dense(len(allowedChars), name='digit2', activation='softmax')(tensor_out),\
              Dense(len(allowedChars), name='digit3', activation='softmax')(tensor_out),\
              Dense(len(allowedChars), name='digit4', activation='softmax')(tensor_out),\
              Dense(len(allowedChars), name='digit5', activation='softmax')(tensor_out)]


In [ ]:
model = Model(inputs=tensor_in, outputs=tensor_out)
model.compile(loss='categorical_crossentropy', optimizer='Adamax', metrics=['accuracy'])
model.summary()

In [ ]:
print("Reading training data...")

train_data = read_train_data(PROCESSED_FOLDER, SIZE)
train_label = read_label_data(LABEL_CSV_FILE, allowedChars, 5)

print("Reading completed")

In [ ]:
filepath = MODEL_FOLDER + "{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_digit4_acc', verbose=1, save_best_only=False, mode='max')
earlystop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, mode='auto')
tensorBoard = TensorBoard(log_dir = 'logs', histogram_freq = 1)
callbacks_list = [tensorBoard, checkpoint]

In [ ]:
print('model loading...')
model = load_model("twse_cnn_model.hdf5")
print('loading completed')

In [ ]:
history = model.fit(train_data, train_label, validation_split=0.2, batch_size=50, epochs=20, verbose=1, callbacks=callbacks_list)

In [ ]:
show_train_history(history, 'digit1_acc', 'val_digit1_acc')